In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = '/content/drive/MyDrive/PakistaniCarDetectionDataset/Splited_dataset/train'
valid_dir = '/content/drive/MyDrive/PakistaniCarDetectionDataset/Splited_dataset/valid'

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)


Found 3096 images belonging to 48 classes.
Found 751 images belonging to 48 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(48, activation='softmax'))  # Assuming you have 48 classes


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size,
    epochs=50
)


Epoch 1/50
96/96 [==============================] - 894s 9s/step - loss: 3.7168 - accuracy: 0.0787 - val_loss: 3.5715 - val_accuracy: 0.0897
Epoch 2/50
96/96 [==============================] - 35s 367ms/step - loss: 3.6122 - accuracy: 0.0891 - val_loss: 3.5561 - val_accuracy: 0.0924
Epoch 3/50
96/96 [==============================] - 35s 362ms/step - loss: 3.5928 - accuracy: 0.0979 - val_loss: 3.5696 - val_accuracy: 0.1209
Epoch 4/50
96/96 [==============================] - 35s 367ms/step - loss: 3.5181 - accuracy: 0.1149 - val_loss: 3.3936 - val_accuracy: 0.1454
Epoch 5/50
96/96 [==============================] - 30s 317ms/step - loss: 3.3636 - accuracy: 0.1498 - val_loss: 3.2758 - val_accuracy: 0.2011
Epoch 6/50
96/96 [==============================] - 31s 319ms/step - loss: 3.1824 - accuracy: 0.1922 - val_loss: 3.0722 - val_accuracy: 0.2418
Epoch 7/50
96/96 [==============================] - 35s 364ms/step - loss: 3.0063 - accuracy: 0.2226 - val_loss: 2.9984 - val_accuracy: 0.2704
E

In [ ]:
loss, accuracy = model.evaluate(valid_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

24/24 [==============================] - 7s 277ms/step - loss: 2.6718 - accuracy: 0.5459
Validation Accuracy: 54.59%


In [ ]:
model.save('car_model_detector.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the trained model
model = load_model('/content/drive/MyDrive/car_model_detector.h5')

# Define directories for loading class indices
train_dir = '/content/drive/MyDrive/PakistaniCarDetectionDataset/Splited_dataset/train'  # Update this with the actual path

# Create a data generator for loading class indices
datagen = ImageDataGenerator(rescale=1./255)

# Initialize the generator to get class indices
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

# Get the class indices from the generator
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

# Function to load and preprocess a single image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(256, 256))  # Resize image to 256x256
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to [0, 1]
    return img_array

# Function to predict the class of the car model
def predict_car_model(img_path):
    img = load_and_preprocess_image(img_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction[0])  # Get the index of the highest probability

   # Debug: Print raw predictions and predicted class
   # print(f'Raw predictions: {prediction}')
   # print(f'Predicted class index: {predicted_class}')

    return predicted_class

# Path to the test image
img_path = '/content/drive/MyDrive/PakistaniCarDetectionDataset/Splited_dataset/train/FAW_XPV/1520.jpg'  # Change to the actual path of the test image

# Predict the class
predicted_class_index = predict_car_model(img_path)
predicted_class_name = class_labels[predicted_class_index]
print(f'Predicted Class: {predicted_class_name}')


Found 3096 images belonging to 48 classes.
1/1 [==============================] - 0s 126ms/step
Predicted Class: FAW_XPV


In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.6 M

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gradio as gr

# Load the trained model
model = load_model('/content/drive/MyDrive/car_model_detector.h5')

# Define directories for loading class indices
train_dir = '/content/drive/MyDrive/PakistaniCarDetectionDataset/Splited_dataset/train'  # Update this with the actual path

# Create a data generator for loading class indices
datagen = ImageDataGenerator(rescale=1./255)

# Initialize the generator to get class indices
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

# Get the class indices from the generator
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

# Function to load and preprocess a single image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(256, 256))  # Resize image to 256x256
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to [0, 1]
    return img_array

# Function to predict the class of the car model
def predict_car_model(img_path):
    img = load_and_preprocess_image(img_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction[0])  # Get the index of the highest probability

    # Debug: Print raw predictions and predicted class
    # print(f'Raw predictions: {prediction}')
    # print(f'Predicted class index: {predicted_class}')

    return class_labels[predicted_class]

# Gradio interface function
def gradio_predict_car_model(image):
    predicted_class_name = predict_car_model(image)
    return predicted_class_name

# Create Gradio interface
interface = gr.Interface(
    fn=gradio_predict_car_model,
    inputs=gr.Image(type='filepath'),  # Change to 'filepath'
    outputs=gr.Textbox(),
    title="Car Model Detector",
    description="Upload an image of a car to detect its model."
)

# Launch the interface
interface.launch()


Found 3096 images belonging to 48 classes.
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://611178e5a8b9cef1ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
